In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [21]:
# import dataset: all audiences AS LCAS
# SELECT *
# FROM `corp-bi-us-prod.rldb.liveramp_customer_accounts` 
# WHERE enabled = true AND customer_link_audience = false
lcas = pd.read_csv('/Users/aschul/Documents/Code_Base/Investigations/20200417_AK_analysis/all_LCAs.csv')

In [ ]:
lcas.info()

### total audiences
21364

In [12]:
lcas.head(2)

,id,customer_id,data_partner_thrift_identifier,display_name,volume_limit,updated_at,updated_by,most_recent_esp_version_reach_stats,created_by,created_at,...,type,automatic_backfill,custom_id_distribution_enabled,data_classification,customer_link_audience,field_encryption_key,import_recency_filter,abilitec_migration_mode,mobile_matching_accuracy_type,partner_link_fast_lane_enabled
0,474731,517576,474731,Cliff Bentz for Congress,NaN,2020-03-24 10:10:23 UTC,1944106,NaN,1944106,2020-03-24 10:10:23 UTC,...,OfflineLiverampCustomerAccount,False,False,0,False,NaN,NaN,2,0.0,False
1,474741,539719,474741,Nerdwallet - Home,NaN,2020-03-24 11:48:55 UTC,2085729,NaN,2085729,2020-03-24 11:48:55 UTC,...,OfflineLiverampCustomerAccount,False,False,0,False,NaN,NaN,2,0.0,False


In [7]:
### Audiences Summary, grouped by Type, enabled = true, customer_link_audience = false
# SELECT type, count(type) AS type_count
# FROM `corp-bi-us-prod.rldb.liveramp_customer_accounts` 
# WHERE enabled = true AND customer_link_audience = false
# GROUP BY type
# ORDER BY type_count DESC

aud_sum = pd.read_csv('/Users/aschul/Documents/Code_Base/Investigations/20200417_AK_analysis/all_LCAs_summary.csv')
aud_sum

,type,type_count
0,OfflineLiverampCustomerAccount,15707
1,OnlineLiverampCustomerAccount,5117
2,CidPinMappingLiverampCustomerAccount,206
3,CidAnonymousIdMappingLiverampCustomerAccount,158
4,MuidOnlyLiverampCustomerAccount,75
5,MuidHeidMappingLiverampCustomerAccount,73
6,MatchLiverampCustomerAccount,45
7,OfflineFulfillmentMappingLiverampCustomerAccount,17
8,IdSyncMappingLiverampCustomerAccount,16


# Data Store vs Non-Data Store Customers, Audiences

In [24]:
# DMS providers - customers
# 804 customers are data providers, defined by having an entry in the dms_provider table and customers.enabled = true
# SELECT dms_providers.customer_id, customers.id
# FROM `corp-bi-us-prod.rldb.dms_providers` AS dms_providers
#   INNER JOIN `corp-bi-us-prod.rldb.customers` AS customers
#     ON dms_providers.customer_id = customers.id
# WHERE customers.status = 1

dms_providers = pd.read_csv('/Users/aschul/Documents/Code_Base/Investigations/20200417_AK_analysis/dms_providers.csv')

dms_providers.count()
dms_providers.head(10)

,customer_id,name
0,263281,Datalab
1,510449,Demo Company
2,39144,Infogroup
3,57384,SMS
4,16851,"test &'"""
5,56444,SafeGraph
6,213651,Relevate
7,140601,ownerIQ
8,509776,Audience Elements APAC
9,513836,Chegg


In [55]:
# Data Store Audiences
# DMS_segments -> LRC_field_definitions -> LR_customer_accounts -> customers 

# SELECT lr_customer_accounts.id AS audience_id, lr_customer_accounts.display_name AS audience_name, customers.name AS customer_name, customers.id AS customer_id, lr_customer_accounts.type
# FROM `corp-bi-us-prod.rldb.dms_segments` AS dms_segments
# INNER JOIN `corp-bi-us-prod.rldb.lrc_field_definitions` AS lrc_field_definitions
#   ON dms_segments.lrc_field_definition_id = lrc_field_definitions.id
# INNER JOIN `corp-bi-us-prod.rldb.liveramp_customer_accounts` AS lr_customer_accounts
#   ON lrc_field_definitions.lr_customer_account_id = lr_customer_accounts.id
# INNER JOIN `corp-bi-us-prod.rldb.customers` AS customers
#   ON lr_customer_accounts.customer_id = customers.id
# WHERE lr_customer_accounts.enabled = true AND lr_customer_accounts.customer_link_audience = false AND customers.status = 1
# GROUP BY lr_customer_accounts.id, lr_customer_accounts.display_name, customers.name, customers.id, lr_customer_accounts.type

dms_audiences = pd.read_csv('/Users/aschul/Documents/Code_Base/Investigations/20200417_AK_analysis/all_DS_non_DS_audiences.csv')
dms_audiences
dms_audiences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   audience_id    1564 non-null   int64 
 1   audience_name  1564 non-null   object
 2   customer_name  1564 non-null   object
 3   customer_id    1564 non-null   int64 
 4   type           1564 non-null   object
dtypes: int64(2), object(3)
memory usage: 61.2+ KB


In [62]:
dms_audiences.head(5)

,audience_id,audience_name,customer_name,customer_id,type
0,301749,Healio - GUID - Onboarding,Healio,518946,OnlineLiverampCustomerAccount
1,485541,Elkay - DataGuru Segments,LinkedIn – Elkay,627691,OnlineLiverampCustomerAccount
2,325639,AudienceBridge - Custom_ST,AudienceBridge,535769,OnlineLiverampCustomerAccount
3,101389,Stirista - Trigger Data,Stirista,510756,OfflineLiverampCustomerAccount
4,287929,Everyday Health - Consumer - data store,Everyday Health - Consumer (Ziff Davis),572639,OfflineLiverampCustomerAccount


In [ ]:
### audience types
#### offline - 15707/21364
#### offline + data store provider - 546

#### online - 5110/21364
#### online + data store provider - 1018

#### how many customers do these audiences represent?

In [60]:
ds_aud_types = dms_audiences['type']

In [61]:
ds_aud_types.value_counts()

OnlineLiverampCustomerAccount     1018
OfflineLiverampCustomerAccount     546
Name: type, dtype: int64

# #2 What % of audiences are email-only audiences?
# Define email-only audiences:
## Conditions:
### 1) one identifier = email | dependency - must normalize AK email labels, identifier email labels
### 2) identifier = AK? false (e.g. CID AK, email identifier)

